In [28]:
from os import chdir
chdir('/Users/lananhnguyen/Desktop/thesis/thesis_code')
from packages.load import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import Pipeline


# 3. Shrinkage method:

In [25]:
HICP_monthly = pd.read_csv('data/preprocessed/hicp_yoy.csv')
HICP_monthly['date'] = pd.to_datetime(HICP_monthly['date'])
HICP_monthly.set_index('date', inplace=True)

In [22]:
HICP_Weights = pd.read_excel('data/HICP_COICOP10s.xlsx', sheet_name='HICP_Weights')
HICP_Weights.head()

,mdates,Rice,Rice.preparations,Flour,Semolina..rye.flour.or.the.like,White.bread,Rye.bread.or.brown.bread,Granary.bread.or.wholemeal.bread,Ready.to.bake.rolls,Fresh.bread.rolls,...,Premium.for.personal.liability.insurance,Premium.for.legal.protection.insurance,Bank.charges,Charge.for.tax.consultant.services,Charge.for.birth.certificate..death.certificate.or.the.like,Passport.fee.or.the.like,Fee.for.lawyer.s.or.notary.s.services,Charge.for.funeral.services.or.cemetery.fee,Classified.advertisement.in.a.newspaper,Visitor.s.tax
0,1996-01-31,0.41,NaN,0.35,0.22,0.68,2.25,1.27,NaN,4.0,...,4.37,1.8,3.15,NaN,NaN,1.1,1.04,0.51,0.44,NaN
1,1996-02-29,0.41,NaN,0.35,0.22,0.68,2.25,1.27,NaN,4.0,...,4.37,1.8,3.15,NaN,NaN,1.1,1.04,0.51,0.44,NaN
2,1996-03-31,0.41,NaN,0.35,0.22,0.68,2.25,1.27,NaN,4.0,...,4.37,1.8,3.15,NaN,NaN,1.1,1.04,0.51,0.44,NaN
3,1996-04-30,0.41,NaN,0.35,0.22,0.68,2.25,1.27,NaN,4.0,...,4.37,1.8,3.15,NaN,NaN,1.1,1.04,0.51,0.44,NaN
4,1996-05-31,0.41,NaN,0.35,0.22,0.68,2.25,1.27,NaN,4.0,...,4.37,1.8,3.15,NaN,NaN,1.1,1.04,0.51,0.44,NaN


In [23]:
HICP_class = pd.read_excel('data/HICP_COICOP10s.xlsx', sheet_name='COICOP_class')
HICP_class.index = ['Group 1', 'Group 2', 'Group 3', 'Group 4']
HICP_class.head()

,Rice,Rice.preparations,Flour,Semolina..rye.flour.or.the.like,White.bread,Rye.bread.or.brown.bread,Granary.bread.or.wholemeal.bread,Ready.to.bake.rolls,Fresh.bread.rolls,Yeast.pastry,...,Premium.for.personal.liability.insurance,Premium.for.legal.protection.insurance,Bank.charges,Charge.for.tax.consultant.services,Charge.for.birth.certificate..death.certificate.or.the.like,Passport.fee.or.the.like,Fee.for.lawyer.s.or.notary.s.services,Charge.for.funeral.services.or.cemetery.fee,Classified.advertisement.in.a.newspaper,Visitor.s.tax
Group 1,Food,Food,Food,Food,Food,Food,Food,Food,Food,Food,...,Non-food,Non-food,Non-food,Non-food,Non-food,Non-food,Non-food,Non-food,Non-food,Non-food
Group 2,Processed food,Processed food,Processed food,Processed food,Processed food,Processed food,Processed food,Processed food,Processed food,Processed food,...,Services,Services,Services,Services,Services,Services,Services,Services,Services,Services
Group 3,bread and cereals,bread and cereals,bread and cereals,bread and cereals,bread and cereals,bread and cereals,bread and cereals,bread and cereals,bread and cereals,bread and cereals,...,miscellaneous services,miscellaneous services,miscellaneous services,miscellaneous services,miscellaneous services,miscellaneous services,miscellaneous services,miscellaneous services,miscellaneous services,miscellaneous services
Group 4,bread and cereals,bread and cereals,bread and cereals,bread and cereals,bread and cereals,bread and cereals,bread and cereals,bread and cereals,bread and cereals,bread and cereals,...,miscellaneous services,miscellaneous services,miscellaneous services,miscellaneous services,miscellaneous services,miscellaneous services,miscellaneous services,miscellaneous services,miscellaneous services,miscellaneous services


## categories here is for food and alcohol:

In [29]:
# HICP for categories:

HICP_food = pd.read_csv('data/preprocessed/food_yoy_infl.csv')
HICP_food['date'] = pd.to_datetime(HICP_food['date'])
HICP_food.set_index('date', inplace=True)
HICP_food.head()

,yoy_rate
date,
1997-01-31,1.430615
1997-02-28,0.995733
1997-03-31,0.283286
1997-04-30,0.423729
1997-05-31,1.272984


In [30]:
# Extract data into groups:
# Full data groups: 

food_col = HICP_class.loc[:, HICP_class.iloc[0, :] == 'Food'].columns
print('Number of items in food group: ', len(food_col))

food_df = HICP_monthly[food_col]
food_df.fillna(0, inplace=True)


# Divide into train and test set:

X_food_train, X_food_test = food_df[food_df.index <= '2015-12-31'].iloc[:-1, :], food_df[food_df.index > '2015-11-30'].iloc[:-1, :]
y_food_train, y_food_test = HICP_food[HICP_food.index <= '2015-12-31'][1:], HICP_food[HICP_food.index > '2015-12-31']

print(f'Training predictor period: {X_food_train.index[0]} to {X_food_train.index[-1]}')
print(f'Training dependent variable period: {y_food_train.index[0]} to {y_food_train.index[-1]}')
print(f'Test predictor period: {X_food_test.index[0]} to {X_food_test.index[-1]}')
print(f'Test dependent variable period: {y_food_test.index[0]} to {y_food_test.index[-1]}')

Number of items in food group:  180
Training predictor period: 1997-01-31 00:00:00 to 2015-11-30 00:00:00
Training dependent variable period: 1997-02-28 00:00:00 to 2015-12-31 00:00:00
Test predictor period: 2015-12-31 00:00:00 to 2022-11-30 00:00:00
Test dependent variable period: 2016-01-31 00:00:00 to 2023-07-31 00:00:00


/var/folders/74/zq2msw1x2fj9j2zyq8njktnc0000gn/T/ipykernel_21660/964387530.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_df.fillna(0, inplace=True)


In [8]:
# 5-fold for train data + 1 horizon:
from shutil import rmtree

from joblib import Memory

# Create a temporary folder to store the transformers of the pipeline
location = "cachedir"
memory = Memory(location=location, verbose=10)

tscv = TimeSeriesSplit(n_splits=5, test_size=6)

pipe_r = Pipeline(
    [
        ("scaling", StandardScaler()),
        ("regression", Ridge()),
    ])

param_grid = {
    'regression__alpha': np.arange(0.9, 1.0, 0.001)
}

grid = GridSearchCV(pipe_r, n_jobs=1, param_grid=param_grid, cv = tscv)
grid.fit(X_food_train, y_food_train)
mean_scores = np.array(grid.cv_results_["mean_test_score"])

In [31]:
print(grid.best_params_)

{'regression__alpha': 0.9990000000000001}


In [65]:
horizons = [1, 2, 3]
max_horizon = 3

forecast_result = pd.DataFrame(columns=['h_1', 'h_2', 'h_3'], index= food_df.iloc[len(X_food_train) - max_horizon +1:, :].index)

for i in range(1, len(X_food_test) + max_horizon): #

    # test data = 3 next observations

    X_train = food_df.iloc[:len(X_food_train) - max_horizon + i +1, :]
    y_train = HICP_food[1:len(X_food_train) - max_horizon + i + 2]
    # y_train is infl of t+1 

    X_test = food_df.iloc[len(X_food_train) - max_horizon +1+ i: len(X_food_train) + 1+ i]

    print(f'Predictor training period: {X_train.index[0]} to {X_train.index[-1]}')
    print(f'Forecast target period: {y_train.index[0]} to {y_train.index[-1]}')
    print(f'Predictor test period: {X_test.index}')

    model = Ridge(alpha=1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test).ravel()
    print(f'Forecast: ')
    print(y_pred)

    if len(y_pred) < 3:
    # Pad with zeros to make it a length of 3
        y_pred = np.pad(y_pred, (0, 3 - len(y_pred)), 'constant')
    forecast_result.iloc[i-1, :] = y_pred

    print('-------------------------------------------------------')


Predictor training period: 1997-01-31 00:00:00 to 2015-10-31 00:00:00
Forecast target period: 1997-02-28 00:00:00 to 2015-11-30 00:00:00
Predictor test period: DatetimeIndex(['2015-11-30', '2015-12-31', '2016-01-31'], dtype='datetime64[ns]', name='date', freq=None)
(3,)
Forecast: 
[ 2.38412775  1.66676772 -0.51997072]
-------------------------------------------------------
Predictor training period: 1997-01-31 00:00:00 to 2015-11-30 00:00:00
Forecast target period: 1997-02-28 00:00:00 to 2015-12-31 00:00:00
Predictor test period: DatetimeIndex(['2015-12-31', '2016-01-31', '2016-02-29'], dtype='datetime64[ns]', name='date', freq=None)
(3,)
Forecast: 
[ 1.03526604 -1.11049984  0.03405384]
-------------------------------------------------------
Predictor training period: 1997-01-31 00:00:00 to 2015-12-31 00:00:00
Forecast target period: 1997-02-28 00:00:00 to 2016-01-31 00:00:00
Predictor test period: DatetimeIndex(['2016-01-31', '2016-02-29', '2016-03-31'], dtype='datetime64[ns]', name='

In [79]:
forecast_df = pd.DataFrame(columns=['h_1', 'h_2', 'h_3'], index= y_food_test[y_food_test.index < '2023-01-31'].index)

forecast_df.iloc[:, 0] = forecast_result.iloc[2:-1, 0]
forecast_df.iloc[:, 1] = forecast_result.iloc[1:-2, 1]
forecast_df.iloc[:, 2] = forecast_result.iloc[:-3, 2]
forecast_df.loc[:, 'real_obs'] = y_food_test[y_food_test.index < '2023-01-31']


In [80]:
forecast_df

,h_1,h_2,h_3,real_obs
date,,,,
2016-01-31,-1.015323,-1.1105,-0.519971,1.209677
2016-02-29,1.976935,0.109354,0.034054,1.103310
2016-03-31,-0.667719,-0.27391,-2.378231,1.402806
2016-04-30,0.141444,-1.418257,-1.119258,1.195219
2016-05-31,0.74602,0.152052,-1.658518,0.896414
...,...,...,...,...
2022-08-31,11.553833,7.14205,6.467057,12.849650
2022-09-30,14.678932,11.617997,7.350677,14.423077
2022-10-31,14.9525,14.271572,11.085574,15.458515


In [24]:
# 5-fold for train data + 1 horizon:
from shutil import rmtree

from joblib import Memory

# Create a temporary folder to store the transformers of the pipeline
location = "cachedir"
memory = Memory(location=location, verbose=10)

tscv = TimeSeriesSplit(n_splits=5, test_size=6)

pipe_l = Pipeline(
    [
        ("scaling", StandardScaler()),
        ("regression", Lasso()),
    ])

param_grid = {
    'regression__alpha': np.arange(0.1, 0.2, 0.001)
}

grid = GridSearchCV(pipe_l, n_jobs=1, param_grid=param_grid, cv = tscv)
grid.fit(X_food_train, y_food_train)


In [26]:
grid.best_params_

{'regression__alpha': 0.1}

---

In [ ]:
services_f = HICP_class.loc[:, HICP_class.iloc[1, :] == 'Services']
energy_f = HICP_class.loc[:, HICP_class.iloc[1, :] == 'Energy']
negi_f = HICP_class.loc[:, HICP_class.iloc[1, :] == 'Non-energy industrial goods (NEIG)']

print('Number of items in food group: ', food_f.shape[1])
print('Number of items in services group: ', services_f.shape[1])
print('Number of items in energy group: ', energy_f.shape[1])
print('Number of items in Non-energy industrial goods (NEIG) group: ', negi_f.shape[1])
print('Total: ', food_f.shape[1] + services_f.shape[1]+ negi_f.shape[1] + energy_f.shape[1])

In [ ]:
food_f = HICP_class.loc[:, HICP_class.iloc[0, :] == 'Food']
services_f = HICP_class.loc[:, HICP_class.iloc[1, :] == 'Services']
energy_f = HICP_class.loc[:, HICP_class.iloc[1, :] == 'Energy']
negi_f = HICP_class.loc[:, HICP_class.iloc[1, :] == 'Non-energy industrial goods (NEIG)']

print('Number of items in food group: ', food_f.shape[1])
print('Number of items in services group: ', services_f.shape[1])
print('Number of items in energy group: ', energy_f.shape[1])
print('Number of items in Non-energy industrial goods (NEIG) group: ', negi_f.shape[1])
print('Total: ', food_f.shape[1] + services_f.shape[1]+ negi_f.shape[1] + energy_f.shape[1])

In [ ]:
# Second groups is for sets that doesn't contain missing values, so subset columns must appear in both df

food = HICP_yoy[HICP_yoy.columns.intersection(food_f.columns)]
services = HICP_yoy[HICP_yoy.columns.intersection(services_f.columns)]
energy = HICP_yoy[HICP_yoy.columns.intersection(energy_f.columns)]
negi = HICP_yoy[HICP_yoy.columns.intersection(negi_f.columns)]

print('Number of items in food group: ', food.shape[1])
print('Number of items in services group: ', services.shape[1])
print('Number of items in energy group: ', energy.shape[1])
print('Number of items in Non-energy industrial goods (NEIG) group: ', negi.shape[1])
print('Total: ', food.shape[1] + services.shape[1]+ negi.shape[1] + energy.shape[1])

In [ ]:
food.head()

In [ ]:
# Divide into train and test set

X_food_train = food[food.index <= '2015-12-31']
X_food_test = food[food.index > '2015-12-31']

#X_food_test = food[(food.index > '2015-12-31') & (food.index <= '2019-12-31')]
#food_df_t2 = food[(food.index > '2019-12-31') & (food.index <= '2021-12-31')]
#food_df_t3 = food[food.index > '2021-12-31']


services_train = services[services.index <= '2015-12-31']
services_t1 = services[(services.index > '2015-12-31') & (services.index <= '2019-12-31')]
services_t2 = services[(services.index > '2019-12-31') & (services.index <= '2021-12-31')]
services_t3 = services[services.index > '2021-12-31']


energy_train = energy[energy.index <= '2015-12-31']
energy_t1 = energy[(energy.index > '2015-12-31') & (energy.index <= '2019-12-31')]
energy_t2 = energy[(energy.index > '2019-12-31') & (energy.index <= '2021-12-31')]
energy_t3 = energy[energy.index > '2021-12-31']


negi_train = negi[negi.index <= '2015-12-31']
negi_t1 = negi[(negi.index > '2015-12-31') & (negi.index <= '2019-12-31')]
negi_t2 = negi[(negi.index > '2019-12-31') & (negi.index <= '2021-12-31')]
negi_t3 = negi[negi.index > '2021-12-31']

In [ ]:
X_food_train = food[food.index <= '2015-12-31']
X_food_test = food[food.index > '2015-12-31']

y_food_train = food_series[food_series.index <= '2015-12-31']
y_food_test = food_series[food_series.index > '2015-12-31']